In [ ]:
import time
import re
import base64
import os
from datetime import datetime
from io import BytesIO
from PIL import Image, ImageDraw
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from twocaptcha import TwoCaptcha
from IPython.display import display

In [ ]:
# Configurar opciones para ejecutar en segundo plano (headless)
firefox_options = Options()
firefox_options.add_argument("--headless")  # Modo sin interfaz gráfica

# Inicializar WebDriver con opciones
driver = webdriver.Firefox(options=firefox_options)

# Acceder a la página (sin abrir ventana)
driver.get("https://login.siat.sat.gob.mx/nidp/app/")
print("WebDriver inicializado.")

# Configurar TwoCaptcha
solver = TwoCaptcha('a04de98e78dd8976ba65b1a7300d461a')

# Obtener la fecha actual en formato DD-MM-YYYY
fecha_actual = datetime.now().strftime("%d-%m-%Y")

# Crear la carpeta si no existe
os.makedirs(fecha_actual, exist_ok=True)

for i in range(3):
    driver.refresh()  # Refrescar la página para obtener un nuevo CAPTCHA

    # Manejar posibles alertas
    try:
        alert = driver.switch_to.alert
        alert.accept()
    except Exception:
        pass

    time.sleep(3)  # Esperar a que se genere el CAPTCHA

    # Obtener el código fuente de la página
    page_source = driver.page_source
    
    # Buscar la imagen CAPTCHA en base64
    match = re.search(r'<img src="data:image/jpeg;base64,([^"]+)"', page_source)

    if match:
        base64_string = match.group(1)
        image_data = base64.b64decode(base64_string)
        image = Image.open(BytesIO(image_data))

        # Guardar temporalmente la imagen en la carpeta
        temp_filename = os.path.join(fecha_actual, "temp_captcha.png")
        image.save(temp_filename)

        # Esperar hasta recibir una respuesta válida de TwoCaptcha
        captcha_code = None
        print("Esperando respuesta de TwoCaptcha...")
        while not captcha_code:  # No se avanza hasta obtener una respuesta
            try:
                result = solver.normal(temp_filename, numeric=4, minLen=6, maxLen=6)  # Resolver el CAPTCHA
                captcha_code = result.get("code", "").upper()  # Convertir a mayúsculas
                print(f"CAPTCHA #{i} resuelto: {captcha_code}")

                # Validar la cadena del CAPTCHA
                if len(captcha_code) != 6 or not captcha_code.isalnum():  # Si la longitud es diferente a 6 o contiene caracteres no alfanuméricos
                    captcha_id = result.get("captchaId")  # Obtener el id del captcha
                    solver.report(captcha_id, False)  # Reportar como incorrecto
                    print("Respuesta reportada.")
            except Exception as e:
                print(f"Error capturado: {e}")
                time.sleep(3)  # Esperar 5 segundos antes de intentar de nuevo

        # Si la cadena es válida, guardar la imagen con el nombre en mayúsculas
        if captcha_code:
            final_filename = os.path.join(fecha_actual, f"{captcha_code}.png")
            image.save(final_filename)

# Cerrar el WebDriver
driver.quit()

# Eliminar el archivo temporal después de finalizar
if os.path.exists(temp_filename):
    os.remove(temp_filename)

Esperando respuesta de TwoCaptcha...
CAPTCHA resuelto: 2K69S2 → Guardado como 26-03-2025\2K69S2.png
Esperando respuesta de TwoCaptcha...
CAPTCHA resuelto: JP942Q → Guardado como 26-03-2025\JP942Q.png
Esperando respuesta de TwoCaptcha...
CAPTCHA resuelto: HW2N6W → Guardado como 26-03-2025\HW2N6W.png


In [ ]:
# Cargar imagen
imagen = Image.open("tipo1.png")

# Definir coordenadas de los recortes (x_inicio, y_inicio, x_fin, y_fin)
rectangulos = [
    (0, 5, 27, 32),  # Primer recorte
    (25, 16, 52, 43),   # Segundo recorte
    (50, 5, 77, 32),
    (75, 16, 102, 43),
    (100, 5, 127, 32),
    (125, 16, 152, 43)
]

# Recortar y guardar cada sección
for i, coords in enumerate(rectangulos):
    recorte = imagen.crop(coords)
    display(recorte)
    # recorte.save(f"recorte_{i+1}.jpg")  # Guardar recorte

# Dibujar rectángulos en la imagen original
dibujar = ImageDraw.Draw(imagen)
for coords in rectangulos:
    dibujar.rectangle(coords, outline="red", width=1)  # Rectángulo rojo

display(imagen)